In [1]:
import requests as rq
import json
import pandas as pd
pd.set_option('display.precision', 4,
              'display.colheader_justify', 'center')
import numpy as np
import warnings
import pytz
import datetime
import time
from IPython.display import clear_output

In [2]:
# Get pro API key
def get_pro_key():
    f = open("/home/vikas/Documents/CG_pro_key.json")
    key_dict = json.load(f)
    return key_dict["key"]

In [3]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_pro_key()
}

In [4]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")

In [5]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

## Get list of networks

In [6]:
networks_list_response = get_response("/onchain/networks",
                                      use_pro, 
                                      "",
                                      PRO_URL)

In [7]:
networks_list_response["data"][0]

{'id': 'eth',
 'type': 'network',
 'attributes': {'name': 'Ethereum', 'coingecko_asset_platform_id': 'ethereum'}}

In [8]:
df_net = pd.DataFrame(networks_list_response["data"])

## Get DEX for a specific network

In [9]:
network = "eth"

dex_list_response = get_response(f"/onchain/networks/{network}/dexes",
                                 use_pro, 
                                 "",
                                 PRO_URL)

In [10]:
df_dex = pd.DataFrame(dex_list_response["data"])

## Create URLs

In [11]:
def get_url(url_type, network, dex = "", pool_address = ""):   

    url_dict = {
        "trending_pools": f"/onchain/networks/{network}/trending_pools",
        "top_pools": f"/onchain/networks/{network}/pools",
        "top_pools_dex": f"/onchain/networks/{network}/dexes/{dex}/pools",
        "specific_pool_dex": f"/onchain/networks/{network}/pools/{pool_address}"
    }

    return url_dict[url_type]

## Get trending pools for a specific network

In [12]:
def collect_response(list_response):    

    response_all = []

    for response in list_response["data"]:
        all_attributes = response["attributes"]
        daily_tx = all_attributes["transactions"]["h24"]
        rel = response["relationships"]
        
        temp_dict = dict(
            pair = all_attributes["name"],
            dex = rel["dex"]["data"]["id"],
            add = all_attributes["address"],
            fdv_usd = all_attributes["fdv_usd"],
            market_cap_usd = all_attributes["market_cap_usd"],
            daily_volume = all_attributes["volume_usd"]["h24"],
            daily_price_change = all_attributes["price_change_percentage"]["h24"],
            daily_buys = daily_tx["buys"],
            daily_sells = daily_tx["sells"],
            daily_buyers = daily_tx["buyers"],
            daily_sellers = daily_tx["sellers"]
        )
        
        response_all.append(temp_dict)

    return response_all

In [13]:
def get_trending_pools(network, sort_by_col):

    target_url = get_url("trending_pools", network)

    trendpool_list_response = get_response(target_url,
                                           use_pro, 
                                           "",
                                           PRO_URL)
    
    trendpool_all = collect_response(trendpool_list_response)    

    return pd.DataFrame(trendpool_all).sort_values(by = [f"{sort_by_col}"],
                                                   ascending = False)

In [14]:
#get_trending_pools("eth", "daily_volume")

## Get top pools for a specific network

In [15]:
def get_top_pools_network(network, sort_by_col):

    target_url = get_url("top_pools", network)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [16]:
#get_top_pools_network("eth", "daily_volume")

## Get top pools for a specific DEX

In [17]:
def get_top_pools_dex(network, dex, sort_by_col):

    target_url = get_url("top_pools_dex", network, dex)

    toppool_list_response = get_response(target_url,
                                         use_pro, 
                                         "",
                                         PRO_URL)

    toppool_all = collect_response(toppool_list_response)   

    return pd.DataFrame(toppool_all).sort_values(by = [f"{sort_by_col}"],
                                                 ascending = False)

In [29]:
get_top_pools_dex("eth", "uniswap_v3", "market_cap_usd")

,pair,dex,add,fdv_usd,market_cap_usd,daily_volume,daily_price_change,daily_buys,daily_sells,daily_buyers,daily_sellers
0,WETH / USDT 0.01%,uniswap_v3,0xc7bbec68d12a0d1830360f8ec58fa599ba1b0e9b,7839659633,7846098328.16383,88063613.8523853,0.25,2789,2446,1432,1666
3,WETH / USDC 0.05%,uniswap_v3,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,7839659633,7846098328.16383,50781474.6535545,0.2,1285,1652,753,1151
7,WETH / USDT 0.05%,uniswap_v3,0x11b815efb8f581194ae79006d24e0d814b7697f6,7839659633,7846098328.16383,11784316.9977573,0.25,781,598,509,411
14,EIGEN / WETH 0.3%,uniswap_v3,0xc2c390c6cd3c4e6c2b70727d35a45e8a072f18ca,6067173229,671426217.992386,4790678.32703423,1.86,173,570,59,398
8,SPX / WETH 0.3%,uniswap_v3,0x7c706586679af2ba6d1a9fc2da9c6af59883fdd3,571859811,571106088.303448,5510508.20446622,-5.44,839,474,569,284
16,VOLT / TITANX 1%,uniswap_v3,0x3f1a36b6c946e406f4295a89ff06a5c7d62f2fe2,43856966,43472958.1045117,2197052.8519651,25.04,531,164,177,113
15,PUFFER / WETH 1%,uniswap_v3,0xc5c9a9ab6403cdba9722463000146c18b504f0ba,382554397,39162246.4557102,153764.914691748,-6.06,57,644,13,621
4,USDC / WETH 0.01%,uniswap_v3,0xe0554a476a092703abdb3ef35c80e0d76d32939f,25868333467,35058510320.2476,41748344.5263432,0.42,1313,1326,904,721
1,Frogs / WETH 1%,uniswap_v3,0x34a14b6f05bd4365f2d159fa0ed46201f2eb4665,5794043,None,5719985.79889539,-35.18,2595,2585,1688,2080
2,Daram / WETH 1%,uniswap_v3,0x7c0d6cbe332dac916944c61956b291a4045f9bfa,22567319,None,12356881.5581372,-39.16,2628,1889,1582,1298


## Get data for a specific pool address

In [19]:
def collect_pool_response(list_response):    

    response = list_response["data"]
    all_attributes = response["attributes"]
    daily_tx = all_attributes["transactions"]["h24"]
    rel = response["relationships"]
        
    response_dict = dict(
        pair = all_attributes["name"],
        dex = rel["dex"]["data"]["id"],
        add = all_attributes["address"],
        fdv_usd = all_attributes["fdv_usd"],
        market_cap_usd = all_attributes["market_cap_usd"],
        daily_volume = all_attributes["volume_usd"]["h24"],
        daily_price_change = all_attributes["price_change_percentage"]["h24"],
        daily_buys = daily_tx["buys"],
        daily_sells = daily_tx["sells"],
        daily_buyers = daily_tx["buyers"],
        daily_sellers = daily_tx["sellers"]
    )

    return response_dict

In [20]:
def get_pool_data(network, dex, pool_address):

    target_url = get_url("specific_pool_dex", network, dex, pool_address)

    pool_list_response = get_response(target_url,
                                      use_pro, 
                                      "",
                                      PRO_URL)

    pool_all = collect_pool_response(pool_list_response)   

    return pool_all

In [51]:
get_pool_data("eth", "uniswap_v3", "0x4e68Ccd3E89f51C3074ca5072bbAC773960dFa36")

{'pair': 'WETH / USDT 0.3%',
 'dex': 'uniswap_v3',
 'add': '0x4e68ccd3e89f51c3074ca5072bbac773960dfa36',
 'fdv_usd': '7832588340',
 'market_cap_usd': '7841413796.96528',
 'daily_volume': '8708301.65072827',
 'daily_price_change': '-0.1',
 'daily_buys': 140,
 'daily_sells': 199,
 'daily_buyers': 112,
 'daily_sellers': 157}

## Uniswap class

In [31]:
from uniswap import Uniswap

In [32]:
address = None        # or None if you're not going to make transactions
private_key = None    # or None if you're not going to make transactions
version = 3           # specify which version of Uniswap to use
provider = "https://rpc.ankr.com/eth"

uniswap = Uniswap(address = address,
                  private_key = private_key,
                  version = version,
                  provider = provider)

In [44]:
# Token contract address
# https://support.uniswap.org/hc/en-us/articles/26757826138637-What-is-a-token-contract-address

eth = "0x0000000000000000000000000000000000000000"
bat = "0x0D8775F648430679A709E98d2b0Cb6250d2887EF"
dai = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
usdt = "0xdAC17F958D2ee523a2206206994597C13D831ec7"

#### Get swap prices

In [50]:
# Returns the amount of ETH you need to pay (in wei) to get X amount of USDT

uniswap.get_price_output(eth, usdt, 7 * 10**6) / (10**18)

No fee set, assuming 0.3%


0.00265403708661202